In [2]:
import matplotlib.pyplot as plt
%matplotlib inline


In [5]:
from astropy.table import Table,QTable

In [4]:
!ls

AnaStarPhotometry.ipynb            processGreyAttenuation_old.py
ComparePhotometry.ipynb            processSkyBackground.py
MakeAstropyTable.ipynb             processStarPhotometry.ecsv
MakePhotometry.ipynb               processStarPhotometry.py
README.md                          qualitycut_amplitudes.py
RelativAtten.ipynb                 qualitycut_fwhm.py
StarElevation.ipynb                qualitycut_fwhm_341.py
bad_obs_23mai19.txt                studyGreyAttenuation.py
cineviewbadspectra.py              tables
cineviewbadspectra_341_342.py      testibin.py
cineviewbadspectrograms.py         view1Dspectra.py
cineviewspectra.py                 view1Dspectra_old.py
cineviewspectra_old.py             view2Dspectra.py
cineviewspectrograms.py            viewAttenuation.py
convertSpectraToPandas.py          viewAttenuation400-800nm.py
correctGreyAttenuation400-800nm.py viewpsf.py
outputtabledir                     viewpsffast.py
predictRayleighAttenuation.py      viewtable.ipynb
processGre

In [12]:
t = Table.read('processStarPhotometry.ecsv', format='ascii.ecsv')

In [15]:
t[:5]

date,airmass,starmag,bkgmag,starmagerr,bkgmagerr,x0,y0,file
object,float64,float64,float64,float64,float64,float64,float64,str72
2019-02-15T22:55:50.730,1.545960928536013,-16.09571996209033,-13.36552151796525,0.0003114148196158889,0.0010531427694586235,416.82351053763523,89.8416717038249,T1M_20190215_225550_730_HD116405_Filtre_None_bin1x1.1_red_spectrum.fits
2019-02-15T22:56:26.007,1.5429357926934,-16.094987030179105,-13.36733463840079,0.0003115473154032512,0.0010522637908860271,417.1428966812357,86.10186965983738,T1M_20190215_225626_007_HD116405_Filtre_None_bin1x1.2_red_spectrum.fits
2019-02-15T22:57:01.284,1.539925136628895,-16.09592430718563,-13.36999514542632,0.0003114314203119332,0.0010509753375414126,416.87375824918206,86.52363002702708,T1M_20190215_225701_284_HD116405_Filtre_None_bin1x1.3_red_spectrum.fits
2019-02-15T22:57:36.562,1.536928795228259,-16.038727952530436,-13.344536563835137,0.0003200994898047642,0.001063369601992767,467.4269565386433,42.90195441951223,T1M_20190215_225736_562_HD116405_Filtre_None_bin1x1.4_red_spectrum.fits
2019-02-15T22:58:11.838,1.533946942906885,-16.04278751991994,-13.32832799651773,0.00031927354369019694,0.0010713366313884422,463.93050592147927,43.06387406816335,T1M_20190215_225811_838_HD116405_Filtre_None_bin1x1.5_red_spectrum.fits


In [29]:
t["airmass"].info.format = '%1.2f'  # for consistent table output
t["starmag"].info.format = '%2.3f'
t["bkgmag"].info.format = '%2.3f'
t["starmagerr"].info.format = '%1.3g'
t["bkgmagerr"].info.format = '%1.3g'
t["x0"].info.format = '%3.0f'
t["y0"].info.format = '%3.0f'

In [31]:
t[:10]

date,airmass,starmag,bkgmag,starmagerr,bkgmagerr,x0,y0,file
object,float64,float64,float64,float64,float64,float64,float64,str72
2019-02-15T22:55:50.730,1.55,-16.096,-13.366,0.000311,0.00105,417,90,T1M_20190215_225550_730_HD116405_Filtre_None_bin1x1.1_red_spectrum.fits
2019-02-15T22:56:26.007,1.54,-16.095,-13.367,0.000312,0.00105,417,86,T1M_20190215_225626_007_HD116405_Filtre_None_bin1x1.2_red_spectrum.fits
2019-02-15T22:57:01.284,1.54,-16.096,-13.370,0.000311,0.00105,417,87,T1M_20190215_225701_284_HD116405_Filtre_None_bin1x1.3_red_spectrum.fits
2019-02-15T22:57:36.562,1.54,-16.039,-13.345,0.00032,0.00106,467,43,T1M_20190215_225736_562_HD116405_Filtre_None_bin1x1.4_red_spectrum.fits
2019-02-15T22:58:11.838,1.53,-16.043,-13.328,0.000319,0.00107,464,43,T1M_20190215_225811_838_HD116405_Filtre_None_bin1x1.5_red_spectrum.fits
2019-02-15T22:58:47.115,1.53,-16.036,-13.346,0.000321,0.00106,463,42,T1M_20190215_225847_115_HD116405_Filtre_None_bin1x1.6_red_spectrum.fits
2019-02-15T22:59:22.399,1.53,-16.042,-13.331,0.000319,0.00107,462,42,T1M_20190215_225922_399_HD116405_Filtre_None_bin1x1.7_red_spectrum.fits
2019-02-15T22:59:57.677,1.53,-16.037,-13.332,0.00032,0.00107,460,38,T1M_20190215_225957_677_HD116405_Filtre_None_bin1x1.8_red_spectrum.fits
2019-02-15T23:00:32.956,1.52,-16.034,-13.327,0.000321,0.00107,457,36,T1M_20190215_230032_956_HD116405_Filtre_None_bin1x1.9_red_spectrum.fits
